In [327]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e11/sample_submission.csv
/kaggle/input/playground-series-s4e11/train.csv
/kaggle/input/playground-series-s4e11/test.csv


In [328]:
import pandas as pd 

Define the train and test variable

In [329]:
df_train = pd.read_csv('/kaggle/input/playground-series-s4e11/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s4e11/test.csv')
test_id = df_test['id']

Determine Howbig is the train data, and the dtype

In [330]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140700 entries, 0 to 140699
Data columns (total 20 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   id                                     140700 non-null  int64  
 1   Name                                   140700 non-null  object 
 2   Gender                                 140700 non-null  object 
 3   Age                                    140700 non-null  float64
 4   City                                   140700 non-null  object 
 5   Working Professional or Student        140700 non-null  object 
 6   Profession                             104070 non-null  object 
 7   Academic Pressure                      27897 non-null   float64
 8   Work Pressure                          112782 non-null  float64
 9   CGPA                                   27898 non-null   float64
 10  Study Satisfaction                     27897 non-null   

Seeing howmuch null value in a columns

In [331]:
df_train.isnull().sum()

id                                            0
Name                                          0
Gender                                        0
Age                                           0
City                                          0
Working Professional or Student               0
Profession                                36630
Academic Pressure                        112803
Work Pressure                             27918
CGPA                                     112802
Study Satisfaction                       112803
Job Satisfaction                          27910
Sleep Duration                                0
Dietary Habits                                4
Degree                                        2
Have you ever had suicidal thoughts ?         0
Work/Study Hours                              0
Financial Stress                              4
Family History of Mental Illness              0
Depression                                    0
dtype: int64

Filling the missing value (this is so bad , can be improve)

In [332]:
from sklearn.impute import SimpleImputer
#Numerical Imputer
numerical_impute = SimpleImputer(fill_value = 0, strategy = 'constant')
numerical_cols = df_train.select_dtypes(include=['float64', 'int64']).columns

#Filter numerical columns with missing values
numerical_cols_with_na = [col for col in numerical_cols if df_train[col].isna().any()]
df_train[numerical_cols_with_na] = numerical_impute.fit_transform(df_train[numerical_cols_with_na])

#Profession Imputer
prof_impute = SimpleImputer(fill_value='Student', strategy ='constant') 
df_train['Profession'] = prof_impute.fit_transform(df_train['Profession'].to_numpy().reshape(-1, 1)).ravel()

#Categorical Imputer
Categorical_impute = SimpleImputer(strategy ='most_frequent') 
categorical_colss = df_train.select_dtypes(include=['object']).columns

#Filling it 
categorical_colss_na = [col for col in categorical_colss if df_train[col].isna().any()]
df_train[categorical_colss_na] = Categorical_impute.fit_transform(df_train[categorical_colss_na])

#Drop co

In [333]:
# Numerical Imputer for df_test
numerical_imputer_test = SimpleImputer(fill_value=0, strategy='constant')
numerical_cols_test = df_test.select_dtypes(include=['float64', 'int64']).columns

# Filter numerical columns with missing values in df_test
numerical_cols_with_na_test = [col for col in numerical_cols_test if df_test[col].isna().any()]
df_test[numerical_cols_with_na_test] = numerical_imputer_test.fit_transform(df_test[numerical_cols_with_na_test])

# Profession Imputer for df_test
profession_imputer_test = SimpleImputer(fill_value='Student', strategy='constant')
df_test['Profession'] = profession_imputer_test.fit_transform(df_test['Profession'].to_numpy().reshape(-1, 1)).ravel()

# Categorical Imputer for df_test
categorical_imputer_test = SimpleImputer(strategy='most_frequent')
categorical_cols_test = df_test.select_dtypes(include=['object']).columns

# Filling missing values in categorical columns for df_test
categorical_cols_na_test = [col for col in categorical_cols_test if df_test[col].isna().any()]
df_test[categorical_cols_na_test] = categorical_imputer_test.fit_transform(df_test[categorical_cols_na_test])

drop the unessecary columns

In [334]:
drop_cols = ['id','Name']
df_test = df_test.drop(drop_cols,axis =1)
df_train = df_train.drop(drop_cols,axis =1)

Label encoding it

In [335]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

categorical_cols = df_train.select_dtypes(include=['object']).columns
categorical_cols_test = df_test.select_dtypes(include=['object']).columns

In [336]:
df_train[categorical_cols] = df_train[categorical_cols].apply(lambda col: LabelEncoder().fit_transform(col))
df_test[categorical_cols_test] = df_test[categorical_cols_test].apply(lambda col: LabelEncoder().fit_transform(col))

In [337]:
y = df_train['Depression']
X = df_train.drop('Depression',axis=1)
X_test = df_test

Make a train test for validating the model

In [338]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(X,y, test_size=0.2,random_state=1)

Using XGBClassifier

In [339]:
from xgboost import XGBClassifier

xgb_params = {
    'learning_rate':0.025,
    'n_estimators':150
}

xgb = XGBClassifier(**xgb_params)

In [340]:
xgb.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.025, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=150, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [341]:
x_preds = xgb.predict(x_valid)

In [342]:
accuracy_score(y_valid,x_preds)

0.9374911158493248

In [343]:
final_mxgb = XGBClassifier(**xgb_params).fit(X,y)

In [344]:
final_xgb = final_mxgb.predict(X_test)

In [345]:
pred = pd.DataFrame({
    'id':test_id,
    'Depression':final_xgb
})

In [346]:
pred

,id,Depression
0,140700,0
1,140701,0
2,140702,0
3,140703,1
4,140704,0
...,...,...
93795,234495,0
93796,234496,1
93797,234497,0
93798,234498,1


In [347]:
pred.to_csv('submission',index=False)